In [1]:
import os
import json
from collections import Counter

import pandas as pd
import sqlalchemy
import dotenv

dotenv.load_dotenv("../.env")

True

In [2]:
engine = sqlalchemy.create_engine(os.getenv("PGCONN"))

In [4]:
# info = pd.read_sql_query("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public'", con=engine)

# dataset = pd.read_sql_table("dataset", con=engine)

# # Get the dataset ID for the dataset we want to query
# dataset_id = dataset.query("name == 'ume-final'")

In [3]:
examples = pd.read_sql_table("example", con=engine)
content = examples["content"].apply(lambda x: json.loads(x))
df = pd.concat([examples.drop("content", axis=1), pd.json_normalize(content)], axis=1)
df["timestamp"] = pd.to_datetime(df["_timestamp"], unit='s')
# Convert dtypes
df = df.replace('', pd.NA)
df["overall"] = df["overall"].astype("Int64")
df["topic"] = df["topic"].astype("Int64")
df["vocabulary"] = df["vocabulary"].astype("Int64")
df["choices"] = df["choices"].astype("Int64")
df

,id,input_hash,task_hash,idx,question,choice_A,choice_B,choice_C,choice_D,correct_answer,...,_view_id,overall,topic,vocabulary,answer,_timestamp,choices,_annotator_id,_session_id,timestamp
0,1,869596046,-482254096,487,A biologist models the population of bacteria ...,The initial population of bacteria in the petr...,The minimum population the petri dish can sustain,The average population of bacteria over the fi...,The maximum population the petri dish can sustain,D,...,blocks,1,3,0,accept,1740425121,2,ume-catherine,ume-catherine,2025-02-24 19:25:21
1,2,1965004864,-2043517965,591,A savings account starts with \$300 and gains ...,8 weeks,12 weeks,24 weeks,30 weeks,B,...,blocks,2,3,2,accept,1740844196,3,ume-final-catherine,ume-final-catherine,2025-03-01 15:49:56
2,3,-759792225,1689464308,487,A biologist models the population of bacteria ...,The initial population of bacteria in the petr...,The minimum population the petri dish can sustain,The average population of bacteria over the fi...,The maximum population the petri dish can sustain,D,...,blocks,1,2,1,accept,1740847740,3,ume-final-catherine,ume-final-catherine,2025-03-01 16:49:00
3,4,-14046686,128649510,508,"A school club with 80 members, equally divided...",$\frac{12}{80}$,$\frac{12}{35}$,$\frac{12}{23}$,$\frac{12}{20}$,B,...,blocks,2,2,3,accept,1740847927,3,ume-final-catherine,ume-final-catherine,2025-03-01 16:52:07
4,5,70955040,-21803035,560,"Company A offers a starting salary of \$30,000...",Year 2,Year 3,Year 4,Year 5,B,...,blocks,2,3,3,accept,1740849204,3,ume-final-catherine,ume-final-catherine,2025-03-01 17:13:24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,446,1000492997,1220658234,683,A student launches a model rocket at a $60^\ci...,50 meters,100 meters,173 meters,200 meters,B,...,blocks,<NA>,<NA>,<NA>,accept,1741230567,<NA>,ume-final-melissa,ume-final-melissa,2025-03-06 03:09:27
446,447,1774954570,-168454859,509,A company produces 200 electronic devices and ...,$3/100$,$3/8$,$2/5$,$5/8$,B,...,blocks,<NA>,<NA>,<NA>,accept,1741230586,<NA>,ume-final-melissa,ume-final-melissa,2025-03-06 03:09:46
447,448,1018722775,-863598565,514,A taxi service charges a base fare plus a fee ...,The base fare charged before the trip begins,The cost per mile charged during the trip,The total cost of the trip when completed,The number of miles traveled in the trip,A,...,blocks,3,3,3,accept,1741230642,3,ume-final-melissa,ume-final-melissa,2025-03-06 03:10:42
448,449,-428529359,485905103,617,The height of a ball above the ground is model...,The initial height of the ball when thrown,The maximum height of the ball,The height of the ball after 1 second,The height of the ball when it hits the ground,A,...,blocks,<NA>,<NA>,<NA>,accept,1741230671,<NA>,ume-final-melissa,ume-final-melissa,2025-03-06 03:11:11


In [7]:
df._annotator_id.value_counts()

_annotator_id
ume-final-jennifer     124
ume-final-stephanie    112
ume-final-melissa      112
ume-final-catherine    101
ume-catherine            1
Name: count, dtype: int64

# Annotator Overlap

In [19]:
# Save samples with less than two complete responses from different annotators
def filter_incomplete_responses(group):
    """Filter out rows with less than two complete responses from
    different annotators."""
    valid_values = [1, 2, 3]
    valid_group = group[
        group["overall"].isin(valid_values) &
        group["topic"].isin(valid_values) &
        group["vocabulary"].isin(valid_values) &
        group["choices"].isin(valid_values)
    ]

    if valid_group["_annotator_id"].nunique() >= 2:
        return False

    return True


for_completion = (
    df
    .groupby("idx")
    .filter(filter_incomplete_responses)
    .drop(
        columns=[
            "_timestamp",
            "_annotator_id",
            "_session_id",
            "_view_id",
        ]
    )
)

print(for_completion.idx.nunique())
display(for_completion.head(5))

72


,id,input_hash,task_hash,idx,question,choice_A,choice_B,choice_C,choice_D,correct_answer,...,display_choice_D,html,_input_hash,_task_hash,overall,topic,vocabulary,answer,choices,timestamp
10,11,1627745375,233784680,559,"At an electronics store, the price of a laptop...",\$758,\$768,\$800,\$850,C,...,$850,"<div class=""question-container"">\n <div class...",1627745375,233784680,<NA>,2,2,accept,3,2025-03-01 19:04:30
12,13,336786636,1999740712,548,A city council wants to assess support for a n...,All apartment dwellers in the downtown area,All homeowners in the downtown area,All residents in the city,All residents in the downtown area,D,...,All residents in the downtown area,"<div class=""question-container"">\n <div class...",336786636,1999740712,3,<NA>,<NA>,accept,<NA>,2025-03-01 19:18:37
13,14,-645976763,-1898733775,558,A student's average score on history quizzes i...,2:03,15:17,3:02,5:02,C,...,5:02,"<div class=""question-container"">\n <div class...",-645976763,-1898733775,1,<NA>,<NA>,accept,<NA>,2025-03-01 19:36:27
17,92,119066446,9736568,565,A cup of coffee cools in a room. Its temperatu...,"The temperature the coffee approaches, $0^{\ci...",The room temperature,The temperature after one hour,The initial temperature of the coffee,B,...,The initial temperature of the coffee,"<div class=""question-container"">\n <div class...",119066446,9736568,3,3,3,accept,3,2025-03-03 19:18:57
38,29,119066446,9736568,565,A cup of coffee cools in a room. Its temperatu...,"The temperature the coffee approaches, $0^{\ci...",The room temperature,The temperature after one hour,The initial temperature of the coffee,B,...,The initial temperature of the coffee,"<div class=""question-container"">\n <div class...",119066446,9736568,1,<NA>,<NA>,accept,<NA>,2025-03-02 00:04:56


In [ ]:
# Find rows with full responses
valid_values = [1, 2, 3]
validation_mask = (
    (df["overall"].isin(valid_values)) &
    (df["topic"].isin(valid_values)) &
    (df["vocabulary"].isin(valid_values)) &
    (df["choices"].isin(valid_values))
)

print(f"Valid annotations: {validation_mask.sum()}")  # 399 Annotations
# 189 items with one or more valid annotations
print(f"Items with valid annotations: {df[validation_mask].idx.nunique()}")

# Create a boolean mask to find rows with any differing choices
diff_mask = (
    (df['question'] != df['question_orig']) |
    (df['choice_A'] != df['choice_A_orig']) |
    (df['choice_B'] != df['choice_B_orig']) |
    (df['choice_C'] != df['choice_C_orig']) |
    (df['choice_D'] != df['choice_D_orig'])
)

# 259 annotations with revisions
print(f"Annotations with revisions: {diff_mask.sum()}")
# 154 items with revisions
print(f"Items with revisions: {df[diff_mask].idx.nunique()}")


def filter_high_agreement_scores(group):
    """Filter out rows with high agreement."""
    if len(group) > 2:
        group = group.sort_values("_timestamp", ascending=True).head(2)
    all_greater_than_one = (
        (all(group["overall"] > 1)) &
        (all(group["topic"] > 1)) &
        (all(group["vocabulary"] > 1)) &
        (all(group["choices"] > 1))
    )
    not_all_three = (
        (not all(group["overall"] == 3)) &
        (not all(group["topic"] == 3)) &
        (not all(group["vocabulary"] == 3)) &
        (not all(group["choices"] == 3))
    )
    return all_greater_than_one and not_all_three


def consolidate_revisions(group):
    """Consolidate multiple revisions into a single row.
    By replacing the original values with the second annotation's revisions.
    """
    row = group.iloc[0].copy()
    if len(group) > 1:
        group = group.sort_values("_timestamp", ascending=True)
        revision_2 = group.iloc[1]
        row["question_orig"] = revision_2["question"]
        row["choice_A_orig"] = revision_2["choice_A"]
        row["choice_B_orig"] = revision_2["choice_B"]
        row["choice_C_orig"] = revision_2["choice_C"]
        row["choice_D_orig"] = revision_2["choice_D"]
    return row


for_adjudication = (
    df[validation_mask & diff_mask]
    .groupby("idx")
    .filter(filter_high_agreement_scores)
    .groupby("idx")
    .apply(consolidate_revisions, include_groups=False)
    .drop(
        columns=[
            "_timestamp",
            "_annotator_id",
            "_session_id",
            "_view_id",
        ]
    )
)

print(f"Rows for adjudication: {len(for_adjudication)}")

Valid annotations: 399
Items with valid annotations: 189
Annotations with revisions: 259
Items with revisions: 154
Rows for adjudication: 9


In [13]:
def filter_good_samples(group):
    """Find samples with unanimous approval."""
    if len(group) > 2:
        group = group.sort_values("_timestamp", ascending=True).head(2)
    all_threes = (
        (all(group["overall"] == 3)) &
        (all(group["topic"] == 3)) &
        (all(group["vocabulary"] == 3)) &
        (all(group["choices"] == 3))
    )
    return all_threes

df[validation_mask].groupby("idx").filter(filter_good_samples)

,id,input_hash,task_hash,idx,question,choice_A,choice_B,choice_C,choice_D,correct_answer,...,_view_id,overall,topic,vocabulary,answer,_timestamp,choices,_annotator_id,_session_id,timestamp
16,91,-363730667,-970634417,626,A line is perpendicular to the line defined by...,$3x - 2y = -1$,$3x - 2y = 1$,$3x + 2y = 19$,$2x + 3y = 21$,A,...,blocks,3,3,3,accept,1741029366,3,ume-final-stephanie,ume-final-stephanie,2025-03-03 19:16:06
17,92,119066446,9736568,565,A cup of coffee cools in a room. Its temperatu...,"The temperature the coffee approaches, $0^{\ci...",The room temperature,The temperature after one hour,The initial temperature of the coffee,B,...,blocks,3,3,3,accept,1741029537,3,ume-final-stephanie,ume-final-stephanie,2025-03-03 19:18:57
43,98,53884816,-632827618,617,The height of a ball above the ground is model...,The initial height of the ball when thrown,The maximum height of the ball,The height of the ball after 1 second,The height of the ball when it hits the ground,A,...,blocks,3,3,3,accept,1741031719,3,ume-final-stephanie,ume-final-stephanie,2025-03-03 19:55:19
56,335,759997307,-429975313,654,A scale model of a building has a volume that ...,$1:64$,$1:16$,$1:4$,$16:1$,B,...,blocks,3,3,3,accept,1741211949,3,ume-final-stephanie,ume-final-stephanie,2025-03-05 21:59:09
100,82,-1164522958,1129261523,542,A savings account starts with \$250 and grows ...,45,46,62,78,A,...,blocks,3,3,3,accept,1741027275,3,ume-final-stephanie,ume-final-stephanie,2025-03-03 18:41:15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418,419,-2130691617,47626093,670,A gardener charges \$40 for equipment rental a...,The hourly labor rate,The number of hours worked,The total cost for one hour of work,The initial equipment rental fee,D,...,blocks,3,3,3,accept,1741227811,3,ume-final-jennifer,ume-final-jennifer,2025-03-06 02:23:31
420,421,-307914360,1379879109,515,The data between daily minutes spent practicin...,$15$,$18$,$30$,$48$,B,...,blocks,2,2,2,accept,1741228040,3,ume-final-jennifer,ume-final-jennifer,2025-03-06 02:27:20
421,422,1380531069,1632893134,549,A water tank contains 200 gallons and leaks at...,$W(d) = -8d + 200$,$W(d) = 8d + 200$,$W(d) = 200d - 8$,$W(d) = -8d$,A,...,blocks,3,3,3,accept,1741228124,3,ume-final-jennifer,ume-final-jennifer,2025-03-06 02:28:44
435,436,-658328051,285332680,710,A plant is $5$ centimeters tall at the beginni...,14,15,16,17,B,...,blocks,3,3,3,accept,1741229609,3,ume-final-melissa,ume-final-melissa,2025-03-06 02:53:29
